In [1]:
import pickle
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from keras.models import *
from keras.layers import *
from keras import optimizers
from keras import regularizers
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
import time
import numpy as np
import pandas as pd
from scipy import interp
from sklearn import metrics
from keras import backend as K
from keras.models import load_model
import csv,os
from sklearn.model_selection import StratifiedKFold
from contextlib import contextmanager

import scipy



Using TensorFlow backend.


In [2]:

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 

#指定第一块GPU可用 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto() 
#不全部占满显存, 按需分配
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

KTF.set_session(sess)

In [3]:
train_path = 'C:/Users/Crow/Desktop/tianchi/security_train.csv'
test_path = 'C:/Users/Crow/Desktop/tianchi/security_test.csv'

In [49]:
DATA_TYPE = {
    'label':np.uint8,
    'file_id':np.uint32,
    'tid':np.uint16,
    'index':np.uint16
}

# PRE-TRAIN-CALSS NUM
OVR_CLASS_NUM = 8

# MULTI-CALSS NUM
CLASS_NUM = 8

# GBT TRAIN ROUND NUM
NUM_ROUND = 1000

# NUMBERS OF LOADING RAW DATA SAMPLES
# IF NONE, LOAD ALL DATA
# ELSE LOAD ${ROWS} DATA
ROWS = 1000000

# 5-Folds CV Param
skf = StratifiedKFold(n_splits=5, random_state=4, shuffle=True)

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

api_vec = TfidfVectorizer(ngram_range=(1,4),
                      min_df=3, max_df=0.9, 
                      strip_accents='unicode', 
                      use_idf=1,smooth_idf=1, sublinear_tf=1)



def tfidfModelTrain(train, test):
    tr_api = train.groupby('file_id')['api'].apply(lambda x:' '.join(x)).reset_index()
    te_api = test.groupby('file_id')['api'].apply(lambda x:' '.join(x)).reset_index()
    tr_api_vec = api_vec.fit_transform(tr_api['api'])
    val_api_vec = api_vec.transform(te_api['api'])
    return (tr_api_vec,val_api_vec)  

def pr(x, y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

def get_mdl(x, y):
    y = y.values
    r = np.log(pr(x,1,y) / pr(x,0,y))
    np.random.seed(0)
    m = LogisticRegression(C=6, dual=True,random_state=0)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

def nblrTrain(tr_tfidf_rlt, te_tfidf_rlt, train):
    label_fold=[]
    preds_fold_lr=[]
    lr_oof = pd.DataFrame()
    preds_te = []
    for fold_i,(tr_index,val_index) in enumerate(skf.split(train, train['label'])):
        if fold_i>=0:
            tr,val = train.iloc[tr_index],train.iloc[val_index]
            x = tr_tfidf_rlt[tr_index,:]
            test_x = tr_tfidf_rlt[val_index,:]        
            preds = np.zeros((len(val), OVR_CLASS_NUM))
            preds_te_i = np.zeros((te_tfidf_rlt.shape[0],OVR_CLASS_NUM))
            labels = [i for i in range(OVR_CLASS_NUM)]
            for i, j in enumerate(labels):
                print('fit', j)
                m,r = get_mdl(x, tr['label'] == j)
                preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
                preds_te_i[:,i] = m.predict_proba(te_tfidf_rlt.multiply(r))[:,1]
            preds_te.append(preds_te_i)
            preds_lr = preds
            lr_oof_i = pd.DataFrame({'file_id':val['file_id']})
            for i in range(OVR_CLASS_NUM):
                lr_oof_i['prob'+str(i)] = preds[:,i]
            lr_oof = pd.concat([lr_oof,lr_oof_i],axis=0)
    
            for i,j in enumerate(preds_lr):
                preds_lr[i] = j/sum(j)
            #log_loss_i = log_loss(val['label'], preds_lr)
            #print(log_loss_i)
            label_fold.append(val['label'].tolist())
            preds_fold_lr.append(preds_lr)
            
            lr_oof = lr_oof.sort_values('file_id')
            preds_te_avg = (np.sum(np.array(preds_te),axis=0) / 5)
            lr_oof_te = pd.DataFrame({'file_id':range(0,te_tfidf_rlt.shape[0])})
            for i in range(OVR_CLASS_NUM):
                lr_oof_te['prob'+str(i)] = preds_te_avg[:,i]
    return (lr_oof, lr_oof_te)

def makeProbFeature(traindata, testdata):
    tr_api_vec, val_api_vec = tfidfModelTrain(traindata, testdata)

In [28]:



# FEATURE ENGINEERING V1
def makeFeature(data, is_train=True):
    '''
    file_cnt: file有多少样本;
    tid_distinct_cnt: file发起了多少线程;
    api_distinct_cnt: file调用了多少不同的API ;
    value_distinct_cnt: file有多少不同的返回值;
    tid_api_cnt_max,tid_api_cnt_min,tid_api_cnt_mean: ","file中的线程调用的 最多/最少/平均 api数目;
    tid_api_distinct_cnt_max, tid_api_distinct_cnt_min, tid_api_distinct_cnt_mean:;
    file中的线程调用的 最多/最少/平均 不同api数目 ;
    '''
    if is_train:
        return_data = data[['file_id', 'label']].drop_duplicates()
    else:
        return_data = data[['file_id']].drop_duplicates()        
    ################################################################################
    feat = data.groupby(['file_id']).tid.count().reset_index(name='file_cnt')
    return_data = return_data.merge(feat, on='file_id', how='left')

    ################################################################################
    feat = data.groupby(['file_id']).agg({'tid':pd.Series.nunique, 'api':pd.Series.nunique}).reset_index()
    feat.columns = ['file_id', 'tid_distinct_cnt', 'api_distinct_cnt']
    return_data = return_data.merge(feat, on='file_id', how='left')
    ################################################################################
    feat_tmp = data.groupby(['file_id', 'tid']).agg({'index':pd.Series.count,'api':pd.Series.nunique}).reset_index()
    feat = feat_tmp.groupby(['file_id'])['index'].agg(['max', 'min', 'mean']).reset_index()
    feat.columns = ['file_id', 'tid_api_cnt_max', 'tid_api_cnt_min', 'tid_api_cnt_mean']
    return_data = return_data.merge(feat, on='file_id', how='left')
    
    feat = feat_tmp.groupby(['file_id'])['api'].agg(['max', 'min', 'mean']).reset_index()
    feat.columns = ['file_id', 'tid_api_distinct_cnt_max','tid_api_distinct_cnt_min', 'tid_api_distinct_cnt_mean']
    return_data = return_data.merge(feat, on='file_id', how='left')
    ################################################################################
   

    return return_data

# FEATURE ENGINEERING V2
def makeFeature_v2(data):
    '''
    api_index_min: api首次出现的index;
    api_cnt: api出现的次数;
    api_rate: api出现的次数占所有api调用次数的比例;
    api_value_equals_0_cnt:   api返回值为0的次数;
    '''
    return_data = data[['file_id']].drop_duplicates()
    
    # 统计file调用api的次数
    tmp = data.groupby(['file_id']).api.count()
    
    # 统计api调用的最小Index
    feat = data.groupby(['file_id', 'api'])['index'].min().reset_index(name='val')
    feat = feat.pivot(index='file_id', columns='api', values='val')
    feat.columns = [ feat.columns[i]+'_index_min' for i in range(feat.shape[1])]
    feat_withFileid = feat.reset_index()
    return_data = return_data.merge(feat_withFileid, on='file_id', how='left')
    # 统计api调用的次数
    feat = data.groupby(['file_id', 'api'])['index'].count().reset_index(name='val')
    feat = feat.pivot(index='file_id', columns='api', values='val')
    feat.columns = [ feat.columns[i]+'_cnt' for i in range(feat.shape[1])]
    feat_withFileid = feat.reset_index()
    return_data = return_data.merge(feat_withFileid, on='file_id', how='left')
    # 统计api调用的比例
    feat_rate = pd.concat([feat, tmp], axis=1)
    feat_rate = feat_rate.apply(lambda x: x/feat_rate.api)
    feat_rate.columns = [ feat_rate.columns[i]+'_rate' for i in range(feat_rate.shape[1])]
    feat_rate_withFileid = feat_rate.reset_index().drop(['api_rate'], axis=1)
    return_data = return_data.merge(feat_rate_withFileid, on='file_id', how='left')

#     # 统计api返回值为0的次数
#     feat = data[data.return_value==0].groupby(['file_id', 'api'])['index'].count().reset_index(name='val')
#     feat = feat.pivot(index='file_id', columns='api', values='val')
#     feat.columns = [ feat.columns[i]+'_value_equals_0_cnt' for i in range(feat.shape[1])]
#     feat_withFileid = feat.reset_index()
#     return_data = return_data.merge(feat_withFileid, on='file_id', how='left')
    
    return return_data 

# FEATURE ENGINEERING V3
def makeFeature_v3(data):
    '''
    api_not0_index_min: api返回值不为0的index的最小值;
    api_not0_index_min_diff: api返回值不为0时最小index和该api出现的最小index的差;
    api_equals0_rate: api返回值为0的次数占该api次数的比例
    '''
    return_data = data[['file_id']].drop_duplicates()
     # 统计api调用的最小Index
    feat_api_min_index = data.groupby(['file_id', 'api'])['index'].min().reset_index(name='min_index')
    feat_api_not0_min_index = data[data.return_value!=0].groupby(['file_id', 'api'])['index'].min().reset_index(name='value_not0_min_index')
    # 统计return_value不为0的最小Index
    feat = feat_api_not0_min_index.pivot(index='file_id', columns='api', values='value_not0_min_index')
    feat.columns = [ feat.columns[i]+'_not0_index_min' for i in range(feat.shape[1])]
    feat_withFileid = feat.reset_index()
    return_data = return_data.merge(feat_withFileid, on='file_id', how='left')   
    # 统计return_value不为0的最小Index和api最小index的差
    feat = feat_api_min_index.merge(feat_api_not0_min_index, on=['file_id', 'api'], how='left')
    feat.loc[:,'api_index_not0_min_diff'] = feat['value_not0_min_index'] - feat['min_index']
    feat = feat.pivot(index='file_id', columns='api', values='api_index_not0_min_diff')
    feat.columns = [ feat.columns[i]+'_not0_index_min_diff' for i in range(feat.shape[1])]
    feat_withFileid = feat.reset_index()
    return_data = return_data.merge(feat_withFileid, on='file_id', how='left')   
    # 统计api返回值为0的次数
    feat = data[data.return_value==0].groupby(['file_id', 'api'])['index'].count().reset_index(name='value_equals0_cnt')
    feat_api_cnt = data.groupby(['file_id', 'api']).return_value.count().reset_index(name='file_api_cnt')
    feat = feat.merge(feat_api_cnt, on=['file_id', 'api'], how='left')
    feat.loc[:,'value_equals0_rate'] = feat['value_equals0_cnt']/(feat['file_api_cnt']*1.0)
    # 统计return_value为0的比例
    feat = feat.pivot(index='file_id', columns='api', values='value_equals0_rate')
    feat.columns = [ feat.columns[i]+'_equals0_rate' for i in range(feat.shape[1])]
    feat_withFileid = feat.reset_index()
    return_data = return_data.merge(feat_withFileid, on='file_id', how='left')   
  
    return return_data         

def makeProbFeature(traindata, testdata):
    tr_api_vec, val_api_vec = tfidfModelTrain(traindata, testdata)

In [9]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.2f}s".format(title, time.time() - t0))
   

In [8]:
traindata = pd.read_csv(train_path, dtype=DATA_TYPE)
testdata = pd.read_csv(test_path, dtype=DATA_TYPE)
print('Train Dataset Length: ', traindata.shape[0])
print('Test Dataset Length: ', testdata.shape[0])  

Train Dataset Length:  89806693
Test Dataset Length:  79288375


In [25]:
traindata

file_id  label                      api   tid  index
0               1      5               LdrLoadDll  2488      0
1               1      5   LdrGetProcedureAddress  2488      1
2               1      5   LdrGetProcedureAddress  2488      2
3               1      5   LdrGetProcedureAddress  2488      3
4               1      5   LdrGetProcedureAddress  2488      4
5               1      5   LdrGetProcedureAddress  2488      5
6               1      5   LdrGetProcedureAddress  2488      6
7               1      5   LdrGetProcedureAddress  2488      7
8               1      5   LdrGetProcedureAddress  2488      8
9               1      5   LdrGetProcedureAddress  2488      9
10              1      5           NtCreateMutant  2488     10
11              1      5                  NtClose  2488     11
12              1      5             NtCreateFile  2488     12
13              1      5              NtWriteFile  2488     13
14              1      5                  NtClose  2488     14
15              1      5   CreateProcessInternalW  2488     15
16              1      5                  NtClose  2488     16
17              1      5                  NtClose  2488     17
18              1      5             LdrUnloadDll  2488     18
19              1      5  NtAllocateVirtualMemory  2488     19
20              1      5      NtFreeVirtualMemory  2488     20
21              1      5  NtAllocateVirtualMemory  2488     21
22              1      5              GetFileType  2488     22
23              1      5              GetFileType  2488     23
24              1      5              GetFileType  2488     24
25              1      5  NtAllocateVirtualMemory  2488     25
26              1      5  NtAllocateVirtualMemory  2488     26
27              1      5  NtAllocateVirtualMemory  2488     27
28              1      5          LdrGetDllHandle  2488     28
29              1      5   LdrGetProcedureAddress  2488     29
...           ...    ...                      ...   ...    ...
89806663    13887      2                  NtClose  2336    593
89806664    13887      2                  NtClose  2336    594
89806665    13887      2                  NtClose  2336    595
89806666    13887      2                  NtClose  2336    596
89806667    13887      2                  NtClose  2336    597
89806668    13887      2                  NtClose  2336    598
89806669    13887      2                  NtClose  2336    599
89806670    13887      2                  NtClose  2336    600
89806671    13887      2                  NtClose  2336    601
89806672    13887      2                  NtClose  2336    602
89806673    13887      2                  NtClose  2336    603
89806674    13887      2                  NtClose  2336    604
89806675    13887      2                  NtClose  2336    605
89806676    13887      2                  NtClose  2336    606
89806677    13887      2                  NtClose  2336    607
89806678    13887      2                  NtClose  2336    608
89806679    13887      2                  NtClose  2336    609
89806680    13887      2          LdrGetDllHandle  2336    610
89806681    13887      2   LdrGetProcedureAddress  2336    611
89806682    13887      2          LdrGetDllHandle  2336    612
89806683    13887      2   LdrGetProcedureAddress  2336    613
89806684    13887      2             LdrUnloadDll  2336    614
89806685    13887      2                NtOpenKey  2336    615
89806686    13887      2          NtQueryValueKey  2336    616
89806687    13887      2                  NtClose  2336    617
89806688    13887      2                  NtClose  2336    618
89806689    13887      2                  NtClose  2336    619
89806690    13887      2                  NtClose  2336    620
89806691    13887      2                  NtClose  2336    621
89806692    13887      2       NtTerminateProcess  2336    622

[89806693 rows x 5 columns]

In [32]:
with timer('GBT Feature Engineering'):
    # MAKE TRAIN DATA FEATURES
    train_base_feature_v1 = makeFeature(traindata, True)
    print('Base Train Data 1: ', train_base_feature_v1.shape)
    train_base_feature_v1.to_csv('data/train_base_features_v1.csv', index=None)

    train_base_feature_v2 = makeFeature_v2(traindata)
    print('Base Train Data 2: ', train_base_feature_v2.shape)
    train_base_feature_v2.to_csv('data/train_base_features_v2.csv', index=None)

#     train_base_feature_v3 = makeFeature_v3(traindata)
#     print('Base Train Data: ', train_base_feature_v3.shape)
#     train_base_feature_v3.to_csv('data/train_base_features_v3.csv', index=None)

    # MAKE TEST DATA FEATURES
    test_base_feature_v1 = makeFeature(testdata, False)
    print('Base Test Data 1: ', test_base_feature_v1.shape)
    test_base_feature_v1.to_csv('data/test_base_features_v1.csv', index=None)

    test_base_feature_v2 = makeFeature_v2(testdata)
    print('Base Test Data 2: ', test_base_feature_v2.shape)
    test_base_feature_v2.to_csv('data/test_base_features_v2.csv', index=None)

#     test_base_feature_v3 = makeFeature_v3(testdata)
#     print('Base Test Data: ', test_base_feature_v3.shape)
#     test_base_feature_v3.to_csv('data/test_base_features_v3.csv', index=None)

    

Base Train Data:  (13887, 11)
Base Train Data:  (13887, 886)
Base Test Data:  (12955, 10)
Base Test Data:  (12955, 895)
GBT Feature Engineering - done in 253.99s


In [54]:
# Provided by 3sigma

with timer('TFIDF and OVR-PROB Feature Engineering'):   
    tr_api_vec, val_api_vec = tfidfModelTrain(traindata, testdata)
    scipy.sparse.save_npz('data/tr_tfidf_rlt.npz', tr_api_vec)
    scipy.sparse.save_npz('data/te_tfidf_rlt.npz', val_api_vec) 

    tr_prob, te_prob = nblrTrain(tr_api_vec, val_api_vec, train_base_feature_v1)
    tr_prob.to_csv('data/tr_lr_oof_prob.csv',index=False)
    te_prob.to_csv('data/te_lr_oof_prob.csv',index=False)

fit 0


C:\Users\Crow\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


fit 1


C:\Users\Crow\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


fit 2
fit 3
fit 4
fit 5
fit 6
fit 7
fit 0
fit 1
fit 2
fit 3
fit 4
fit 5
fit 6
fit 7
fit 0
fit 1
fit 2
fit 3
fit 4
fit 5
fit 6
fit 7
fit 0
fit 1
fit 2
fit 3
fit 4
fit 5
fit 6
fit 7
fit 0
fit 1
fit 2
fit 3
fit 4
fit 5
fit 6
fit 7
TFIDF and OVR-PROB Feature Engineering - done in 754.93s


file_id         prob0     prob1     prob2     prob3     prob4  \
0            0  7.348098e-03  0.000821  0.962587  0.007986  0.000426   
1            1  8.373348e-01  0.000847  0.001441  0.000725  0.000310   
2            2  9.999794e-01  0.000334  0.000102  0.000438  0.000470   
3            3  3.874701e-02  0.000614  0.003378  0.010014  0.001355   
4            4  9.812372e-01  0.000598  0.001858  0.001633  0.000355   
5            5  2.579256e-03  0.001892  0.007412  0.001100  0.000751   
6            6  2.482239e-04  0.000653  0.001672  0.752301  0.005698   
7            7  9.253864e-01  0.001850  0.013173  0.004524  0.000441   
8            8  7.484188e-01  0.000915  0.003474  0.003420  0.000604   
9            9  7.848235e-01  0.000851  0.009195  0.005637  0.000095   
10          10  1.502479e-02  0.001180  0.004459  0.002159  0.000132   
11          11  7.303114e-01  0.001932  0.008329  0.002834  0.000538   
12          12  9.932137e-01  0.000745  0.000866  0.001163  0.000405   
13          13  8.585272e-01  0.000967  0.008138  0.003133  0.000405   
14          14  9.290608e-01  0.000898  0.057632  0.002022  0.000169   
15          15  9.983385e-01  0.000610  0.001667  0.001026  0.000186   
16          16  2.557908e-06  0.000732  0.006476  0.001389  0.017908   
17          17  9.760814e-01  0.005336  0.002420  0.012276  0.000648   
18          18  9.552778e-01  0.000887  0.004843  0.003328  0.000803   
19          19  9.999979e-01  0.000252  0.000058  0.000209  0.000400   
20          20  9.279677e-01  0.001339  0.006437  0.000239  0.000368   
21          21  1.905648e-03  0.000628  0.003480  0.070302  0.000212   
22          22  2.491203e-02  0.000721  0.486376  0.000919  0.000364   
23          23  9.700662e-01  0.001026  0.004650  0.003169  0.000527   
24          24  1.818628e-05  0.000292  0.024539  0.000276  0.000297   
25          25  5.948188e-06  0.000598  0.001373  0.004905  0.000077   
26          26  1.498978e-02  0.013399  0.007663  0.000993  0.000710   
27          27  8.405402e-01  0.000897  0.005016  0.005661  0.003524   
28          28  1.980118e-03  0.002470  0.001796  0.008078  0.000187   
29          29  5.179572e-05  0.000742  0.002071  0.588186  0.021351   
...        ...           ...       ...       ...       ...       ...   
12925    12925  9.941699e-01  0.000702  0.003027  0.002273  0.000302   
12926    12926  7.957036e-01  0.000692  0.000894  0.000690  0.000247   
12927    12927  2.319339e-06  0.001393  0.003150  0.001860  0.000437   
12928    12928  1.796599e-02  0.001970  0.004677  0.001829  0.000752   
12929    12929  4.503852e-04  0.002116  0.016382  0.001312  0.000516   
12930    12930  2.049918e-03  0.910003  0.001819  0.002833  0.000210   
12931    12931  1.319469e-03  0.991715  0.001087  0.009667  0.000327   
12932    12932  9.668223e-01  0.000897  0.001540  0.001902  0.001007   
12933    12933  7.764331e-01  0.000917  0.006776  0.003330  0.000643   
12934    12934  4.928545e-04  0.000753  0.002074  0.151891  0.392312   
12935    12935  8.487188e-01  0.001027  0.006521  0.004040  0.000557   
12936    12936  8.187120e-03  0.959835  0.001333  0.001792  0.000275   
12937    12937  1.346802e-08  0.000403  0.002751  0.000388  0.001206   
12938    12938  6.469049e-04  0.000570  0.002516  0.005648  0.000563   
12939    12939  8.350758e-01  0.001042  0.001621  0.001585  0.001091   
12940    12940  9.999933e-01  0.000264  0.000106  0.000295  0.000375   
12941    12941  1.356536e-03  0.000949  0.066474  0.002810  0.000175   
12942    12942  1.578423e-02  0.000860  0.005056  0.051141  0.004293   
12943    12943  9.999976e-01  0.000240  0.000050  0.000264  0.000577   
12944    12944  9.751429e-01  0.000597  0.001578  0.001568  0.000436   
12945    12945  6.336315e-07  0.000882  0.001421  0.015206  0.000977   
12946    12946  9.750558e-01  0.000654  0.005060  0.001634  0.000464   
12947    12947  9.936761e-01  0.003506  0.002294  0.001251  0.000223   
12948    12948  1.720465e-03  0.943800  0.00109

In [31]:
train_base_feature_v1

file_id  label  file_cnt  tid_distinct_cnt  api_distinct_cnt  \
0            1      5      6786                11               116   
1            2      2       816                 4                30   
2            3      0       463                 1                42   
3            4      0      2046                 9                51   
4            5      0     10002                 6                65   
5            6      0      5001                 4                62   
6            7      0      5282                13                83   
7            8      0       118                 1                24   
8            9      0     18028                32               107   
9           10      0       205                 3                29   
10          11      0       110                 2                23   
11          12      1       480                 6                45   
12          13      5     12547                15                73   
13          14      0      5073                 4                70   
14          15      7      5447                26                70   
15          16      5     13879                26                93   
16          17      0       231                 1                25   
17          18      2       260                 1                29   
18          19      0       262                 2                24   
19          20      7     13679                46                85   
20          21      5      5001                14               112   
21          22      0      1623                 3                62   
22          23      0      5001                 1                57   
23          24      0        49                 1                11   
24          25      3     18728                85                95   
25          26      7     40348                77                66   
26          27      5      7546                29                72   
27          28      0        20                 1                10   
28          29      3      8681                34                83   
29          30      0        23                 1                11   
...        ...    ...       ...               ...               ...   
13857    13858      6      2993                21                53   
13858    13859      2      1324                 4                34   
13859    13860      2        64                 4                 9   
13860    13861      1      1413                15                45   
13861    13862      5         2                 2                 1   
13862    13863      5      5409                 6                32   
13863    13864      2      2984                 4                29   
13864    13865      0       811                 3                52   
13865    13866      5      1557                 4                45   
13866    13867      7       432                 7                32   
13867    13868      5     24232                28                97   
13868    13869      0       139                 3                25   
13869    13870      2       268                 1                30   
13870    13871      7      2020                10                75   
13871    13872      0      3048                 5                71   
13872    13873      0       125                 2                14   
13873    13874      0        48                 3                18   
13874    13875      0       892                 3                52   
13875    13876      0     29458                25               110   
13876    13877      2       654                 6                56   
13877    13878      0       110                 1                16   
13878    13879      5         4                 2                 2   
13879    13880      2     11415                23                69   
13880    13881      7      8982                44                84   
13881    13882      1     20939

In [56]:
# 特征融合，训练
with timer('LOAD FEATURE DATA'):
    # LOAD FEATURE V1
    train_1 = pd.read_csv('data/train_base_features_v1.csv') 
    test_1 = pd.read_csv('data/test_base_features_v1.csv')
    # LOAD FEATURE V2
    train_2 = pd.read_csv('data/train_base_features_v2.csv')
    test_2 = pd.read_csv('data/test_base_features_v2.csv')

    interaction_feat = train_2.columns[train_2.columns.isin(test_2.columns.values)].values
    train_2 = train_2[interaction_feat]
    test_2 = test_2[interaction_feat]
    
   

    # MERGE ALL FEATURES
    train = train_1.merge(train_2, on=['file_id'], how='left')
    test = test_1.merge(test_2,on=['file_id'], how='left')
   
    print('[TRAIN SIZE]: ', train.shape)
    print('[TEST SIZE]: ', test.shape)

    # TRAIN DATA PREPARE
    X = train.drop(['file_id','label'], axis=1)
    y = train['label']
    print('[TRAIN FEATURE SIZE]: ', X.shape)
    print('[TRAIN LABEL DISTRIBUTION]: ')
    print(y.value_counts()) 

[TRAIN SIZE]:  (13887, 887)
[TEST SIZE]:  (12955, 886)
[TRAIN FEATURE SIZE]:  (13887, 885)
[TRAIN LABEL DISTRIBUTION]: 
0    4978
5    4289
7    1487
2    1196
3     820
6     515
1     502
4     100
Name: label, dtype: int64
LOAD FEATURE DATA - done in 3.07s


In [57]:
with timer('ADD ONE_VS_REST PROB'):
    extra_feat_val = pd.read_csv('data/tr_lr_oof_prob.csv')
    extra_feat_test = pd.read_csv('data/te_lr_oof_prob.csv')
    prob_list = ['prob'+str(i) for i in range(OVR_CLASS_NUM)]
    X_extra = pd.concat(
        [X, extra_feat_val[prob_list]], axis=1)
    test_extra = pd.concat(
        [test, extra_feat_test[prob_list]],axis=1)

ADD ONE_VS_REST PROB - done in 0.28s


In [59]:
X

file_cnt  tid_distinct_cnt  api_distinct_cnt  tid_api_cnt_max  \
0          6786                11               116             4744   
1           816                 4                30              204   
2           463                 1                42              463   
3          2046                 9                51              971   
4         10002                 6                65             4921   
5          5001                 4                62             4899   
6          5282                13                83             3626   
7           118                 1                24              118   
8         18028                32               107             4415   
9           205                 3                29              181   
10          110                 2                23               55   
11          480                 6                45              182   
12        12547                15                73             2716   
13         5073                 4                70             3657   
14         5447                26                70             1033   
15        13879                26                93             1491   
16          231                 1                25              231   
17          260                 1                29              260   
18          262                 2                24              131   
19        13679                46                85             3148   
20         5001                14               112             4047   
21         1623                 3                62             1578   
22         5001                 1                57             5001   
23           49                 1                11               49   
24        18728                85                95             3070   
25        40348                77                66             5001   
26         7546                29                72             1029   
27           20                 1                10               20   
28         8681                34                83             2541   
29           23                 1                11               23   
...         ...               ...               ...              ...   
13857      2993                21                53              474   
13858      1324                 4                34              331   
13859        64                 4                 9               16   
13860      1413                15                45              182   
13861         2                 2                 1                1   
13862      5409                 6                32             1491   
13863      2984                 4                29              781   
13864       811                 3                52              773   
13865      1557                 4                45              398   
13866       432                 7                32              334   
13867     24232                28                97             4893   
13868       139                 3                25              116   
13869       268                 1                30              268   
13870      2020                10                75             1034   
13871      3048                 5                71             2974   
13872       125                 2                14              118   
13873        48                 3                18               24   
13874       892                 3                52              847   
13875     29458                25               110             5001   
13876       654                 6                56              421   
13877       110                 1                16              110   
13878         4                 2                 2                2   
13879     11415                23                69             4293   
13880      8982                44              

In [58]:
X_extra

file_cnt  tid_distinct_cnt  api_distinct_cnt  tid_api_cnt_max  \
0          6786                11               116             4744   
1           816                 4                30              204   
2           463                 1                42              463   
3          2046                 9                51              971   
4         10002                 6                65             4921   
5          5001                 4                62             4899   
6          5282                13                83             3626   
7           118                 1                24              118   
8         18028                32               107             4415   
9           205                 3                29              181   
10          110                 2                23               55   
11          480                 6                45              182   
12        12547                15                73             2716   
13         5073                 4                70             3657   
14         5447                26                70             1033   
15        13879                26                93             1491   
16          231                 1                25              231   
17          260                 1                29              260   
18          262                 2                24              131   
19        13679                46                85             3148   
20         5001                14               112             4047   
21         1623                 3                62             1578   
22         5001                 1                57             5001   
23           49                 1                11               49   
24        18728                85                95             3070   
25        40348                77                66             5001   
26         7546                29                72             1029   
27           20                 1                10               20   
28         8681                34                83             2541   
29           23                 1                11               23   
...         ...               ...               ...              ...   
13857      2993                21                53              474   
13858      1324                 4                34              331   
13859        64                 4                 9               16   
13860      1413                15                45              182   
13861         2                 2                 1                1   
13862      5409                 6                32             1491   
13863      2984                 4                29              781   
13864       811                 3                52              773   
13865      1557                 4                45              398   
13866       432                 7                32              334   
13867     24232                28                97             4893   
13868       139                 3                25              116   
13869       268                 1                30              268   
13870      2020                10                75             1034   
13871      3048                 5                71             2974   
13872       125                 2                14              118   
13873        48                 3                18               24   
13874       892                 3                52              847   
13875     29458                25               110             5001   
13876       654                 6                56              421   
13877       110                 1                16              110   
13878         4                 2                 2                2   
13879     11415                23                69             4293   
13880      8982                44              

In [60]:
from sklearn.metrics import log_loss
import xgboost 
def xgbMultiTrain(X_train, X_val, y_train, y_val, test, num_round):

    # multi-cls model
    dtrain = xgboost.DMatrix(X_train, y_train)      
    dval = xgboost.DMatrix(X_val, y_val)    
    dtest = xgboost.DMatrix(test.drop(['file_id'], axis=1))
    watchlist = [(dtrain, 'train'), (dval, 'val')]
    model = xgboost.train(xgb_params_multi,
                      dtrain, 
                      num_round, 
                      evals=watchlist, 
                      early_stopping_rounds=100,
                      verbose_eval=100
                     )
    p_val = pd.DataFrame(model.predict(dval, ntree_limit=model.best_iteration), index=X_val.index)  
    p_test = pd.DataFrame(model.predict(dtest, ntree_limit=model.best_iteration), index=test.index)
    return (model, p_val, p_test)

In [61]:
xgb_params_multi = {'objective':'multi:softprob',
              'num_class':CLASS_NUM,
              'eta': 0.04,  
              'max_depth':6,
              'subsample':0.9,
              'colsample_bytree':0.7,
              'lambda': 2,
              'alpha': 2,
              'gamma': 1,
              'scale_pos_weight':20,
              'eval_metric': 'mlogloss',
              'silent':0,
              'seed':149
}

with timer('5-Fold Multi-Class Model Training'):
    # Variables
    logloss_rlt = []
    p_val_all = pd.DataFrame()
    p_test_all = pd.DataFrame(np.zeros((test.shape[0], 8)))

    # Start 5-fold CV
    for fold_i,(tr_index,val_index) in enumerate(skf.split(X_extra, y)):
        print('FOLD -', fold_i, ' Start...')
        # Prepare train, val dataset
        X_train, X_val = X_extra.iloc[tr_index,:], X_extra.iloc[val_index,:]
        y_train, y_val = y[tr_index], y[val_index]
        # Train model
        model, p_val, p_test = xgbMultiTrain(X_train, X_val, y_train, y_val, test_extra, NUM_ROUND)
        # Evaluate Model and Concatenate Val-Prediction
        m_log_loss = log_loss(y_val, p_val)
        print('----------------log_loss : ', m_log_loss, ' ---------------------')
        logloss_rlt = logloss_rlt + [m_log_loss]
        truth_prob_df = pd.concat([y_val, p_val], axis=1)
        p_val_all = pd.concat([p_val_all, truth_prob_df], axis=0)
        # Predict Test Dataset
        p_test_all = p_test_all + 0.2*p_test

FOLD - 0  Start...


C:\Users\Crow\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-mlogloss:1.95182	val-mlogloss:1.95513
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 100 rounds.
[100]	train-mlogloss:0.204095	val-mlogloss:0.307171
[200]	train-mlogloss:0.128967	val-mlogloss:0.276757
[300]	train-mlogloss:0.107735	val-mlogloss:0.275974
Stopping. Best iteration:
[261]	train-mlogloss:0.113695	val-mlogloss:0.275767

----------------log_loss :  0.2757986019731067  ---------------------
FOLD - 1  Start...


C:\Users\Crow\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-mlogloss:1.95081	val-mlogloss:1.95324
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 100 rounds.
[100]	train-mlogloss:0.202624	val-mlogloss:0.309637
[200]	train-mlogloss:0.12784	val-mlogloss:0.283657
[300]	train-mlogloss:0.107341	val-mlogloss:0.285671
Stopping. Best iteration:
[201]	train-mlogloss:0.127463	val-mlogloss:0.283655

----------------log_loss :  0.2836570692156147  ---------------------
FOLD - 2  Start...


...

[0]	train-mlogloss:1.94803	val-mlogloss:1.95091
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 100 rounds.
[100]	train-mlogloss:0.204901	val-mlogloss:0.298857
[200]	train-mlogloss:0.131693	val-mlogloss:0.270681
[300]	train-mlogloss:0.110533	val-mlogloss:0.270353
Stopping. Best iteration:
[275]	train-mlogloss:0.11385	val-mlogloss:0.26977

----------------log_loss :  0.2697872511416554  ---------------------
FOLD - 3  Start...


C:\Users\Crow\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-mlogloss:1.95004	val-mlogloss:1.95336
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 100 rounds.
[100]	train-mlogloss:0.201502	val-mlogloss:0.318587
[200]	train-mlogloss:0.126617	val-mlogloss:0.292507
[300]	train-mlogloss:0.105668	val-mlogloss:0.293149
Stopping. Best iteration:
[208]	train-mlogloss:0.123719	val-mlogloss:0.292371

----------------log_loss :  0.2923964792926278  ---------------------
FOLD - 4  Start...


C:\Users\Crow\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-mlogloss:1.94911	val-mlogloss:1.95237
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 100 rounds.
[100]	train-mlogloss:0.204265	val-mlogloss:0.296159
[200]	train-mlogloss:0.130907	val-mlogloss:0.267088
[300]	train-mlogloss:0.109614	val-mlogloss:0.266308
Stopping. Best iteration:
[269]	train-mlogloss:0.114105	val-mlogloss:0.266148

----------------log_loss :  0.26617073338084457  ---------------------
5-Fold Multi-Class Model Training - done in 792.41s


In [62]:
with timer('Evaluation'):
    print('[LOGLOSS]: ', logloss_rlt)
    print('[LOGLOSS MEAN]: ', log_loss(p_val_all.iloc[:,0], p_val_all.iloc[:,1:]))
    #print('[LOGLOSS STD]: ', np.std(logloss_rlt))
    print('[LOGLOSS STD]: ', pd.Series(logloss_rlt).std())
    feat_imp = pd.Series(model.get_fscore()).sort_values(ascending=False)
    print('[TOP20 IMPORTANT FEATURES(5TH-FOLD MODEL)]: ')
    print(feat_imp[:20])
    


[LOGLOSS]:  [0.2757986019731067, 0.2836570692156147, 0.2697872511416554, 0.2923964792926278, 0.26617073338084457]
[LOGLOSS MEAN]:  0.27756309685384123
[LOGLOSS STD]:  0.0106128329420466
[TOP20 IMPORTANT FEATURES(5TH-FOLD MODEL)]: 
prob0                               967
prob5                               857
prob2                               825
prob7                               814
prob6                               707
prob3                               576
prob1                               552
prob4                               518
NtClose_index_min                   391
NtClose_cnt_rate                    312
LdrLoadDll_cnt_rate                 284
LdrGetProcedureAddress_cnt_rate     273
NtAllocateVirtualMemory_cnt_rate    250
tid_api_cnt_mean                    239
NtFreeVirtualMemory_cnt_rate        237
LdrGetDllHandle_index_min           232
NtClose_cnt                         224
NtDelayExecution_cnt                221
LdrGetProcedureAddress_cnt          212
SetUnhand

In [63]:
with timer('SUBMIT CHECK'):
    rlt = pd.concat([test['file_id'], p_test_all], axis=1)    
    rlt.columns = ['file_id'] + prob_list
    check_flag = all(rlt.iloc[:,1:].sum(axis=1)-1<1e-6)
    if check_flag:
        print('RESULT IS OK...')
        rlt.to_csv('submit/rlt_TEST.csv', index=None)
        print('RESULT SAVED...')
    else:
        print('RESULT IS WRONG!')

RESULT IS OK...
RESULT SAVED...
SUBMIT CHECK - done in 0.21s


In [78]:
# 模型融合,result 
path1 = 'result_dpcnn_st3.csv'
result1 = pd.read_csv(path1)
path2 = 'result_textcnn_TextCNN_1st.csv'
result2 = pd.read_csv(path2)
path3 = 'submit/rlt_TEST.csv'
result3 = pd.read_csv(path3)
path4 = 'result_attention_1st.csv'
result4 = pd.read_csv(path4)

In [66]:
result1

file_id     prob0     prob1     prob2     prob3     prob4     prob5  \
0            1  0.008055  0.009951  0.737807  0.020309  0.009261  0.004225   
1            2  0.715051  0.002488  0.042425  0.003312  0.000741  0.086751   
2            3  0.978289  0.001302  0.014126  0.000658  0.000397  0.001195   
3            4  0.059802  0.005068  0.019485  0.031353  0.004978  0.042002   
4            5  0.932647  0.003532  0.023398  0.001142  0.000284  0.019628   
5            6  0.005346  0.009382  0.002329  0.001398  0.000267  0.977771   
6            7  0.002370  0.000218  0.004105  0.469073  0.010207  0.188545   
7            8  0.917105  0.004356  0.043307  0.002641  0.001423  0.008269   
8            9  0.401563  0.004529  0.024297  0.020560  0.002375  0.066554   
9           10  0.767540  0.004385  0.083694  0.001050  0.000346  0.078002   
10          11  0.003335  0.001301  0.004430  0.000376  0.000086  0.980143   
11          12  0.926437  0.006574  0.030244  0.002195  0.001335  0.020015   
12          13  0.981891  0.000244  0.003596  0.000026  0.000045  0.009383   
13          14  0.883937  0.002016  0.056419  0.002711  0.000409  0.026300   
14          15  0.821183  0.002263  0.052229  0.002250  0.000682  0.014117   
15          16  0.969060  0.000253  0.001274  0.000143  0.000043  0.023053   
16          17  0.000010  0.000043  0.003923  0.006220  0.000107  0.974891   
17          18  0.961905  0.002687  0.004155  0.000109  0.000045  0.018070   
18          19  0.938545  0.005007  0.031043  0.003388  0.001378  0.007855   
19          20  0.959721  0.001299  0.020867  0.002418  0.000452  0.002154   
20          21  0.970163  0.000087  0.015083  0.000054  0.000034  0.011082   
21          22  0.000731  0.000280  0.003835  0.143529  0.000917  0.025927   
22          23  0.005030  0.000880  0.629273  0.018053  0.001531  0.102378   
23          24  0.859446  0.011654  0.050072  0.009940  0.002671  0.032160   
24          25  0.000129  0.000048  0.008107  0.000386  0.000016  0.987983   
25          26  0.000129  0.000037  0.000579  0.000740  0.000024  0.988709   
26          27  0.005423  0.009432  0.002362  0.001407  0.000271  0.977567   
27          28  0.391435  0.022279  0.044040  0.085850  0.017736  0.115551   
28          29  0.000237  0.000193  0.000480  0.009030  0.000407  0.002812   
29          30  0.001162  0.000151  0.001946  0.580281  0.008997  0.170863   
...        ...       ...       ...       ...       ...       ...       ...   
12925    12926  0.987527  0.000106  0.003169  0.000015  0.000024  0.006746   
12926    12927  0.674200  0.001099  0.010083  0.002813  0.001002  0.257892   
12927    12928  0.002685  0.008391  0.003900  0.004419  0.000570  0.971945   
12928    12929  0.005347  0.009365  0.002323  0.001394  0.000267  0.977804   
12929    12930  0.001786  0.000219  0.115086  0.000864  0.000160  0.870440   
12930    12931  0.007117  0.816588  0.012262  0.006723  0.000108  0.033409   
12931    12932  0.002364  0.936466  0.011653  0.003954  0.000134  0.011893   
12932    12933  0.941136  0.002057  0.003258  0.000443  0.000211  0.017751   
12933    12934  0.823026  0.013600  0.074973  0.002886  0.000657  0.041478   
12934    12935  0.011026  0.001431  0.014958  0.284196  0.034576  0.080189   
12935    12936  0.404731  0.027194  0.129863  0.042829  0.009135  0.064204   
12936    12937  0.006222  0.872915  0.008901  0.005095  0.000090  0.021603   
12937    12938  0.000074  0.000014  0.000311  0.000434  0.000044  0.992507   
12938    12939  0.000830  0.000435  0.006894  0.005687  0.000185  0.927090   
12939    12940  0.838276  0.002342  0.012784  0.000573  0.000347  0.121602   
12940    12941  0.974928  0.000755  0.004700  0.000705  0.000145  0.004945   
12941    12942  0.000220  0.000039  0.002113  0.001139  0.000119  0.959647   
12942    12943  0.111421  0.035540  0.031861  0.124295  0.021778  0.208217   
12943    12944  0.979544  0.001180  0.013563  0.000555  0.000361  0.001128   
12944    12945  0.952349  0.0

In [79]:
result = result1 + result2 + result3 + result4

In [80]:
result /= 4.0

In [82]:
result.to_csv('textcnn_attention_dpcnn_xgboost_merge.csv')